# 4. Análise Preditiva Simplificada

This notebook demonstrates a simplified machine learning process to predict exam scores, as outlined in the "Análise Preditiva" section of the portfolio.

In [ ]:
# If we saved the cleaned data, we could load it here instead:
# df = pd.read_parquet('cleaned_student_data.parquet')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Feature Selection and Preprocessing

Based on EDA and HTML feature importance chart, let's select features. The HTML mentions:
`study_hours_per_day`, `attendance_percentage`, `mental_health_rating`, `sleep_hours`, `internet_quality`, `parental_education_level`.
We'll use these and a few more common sense ones.

In [ ]:
# Drop student_id if it's still there
if 'student_id' in df.columns:
    df_model = df.drop('student_id', axis=1).copy()
else:
    df_model = df.copy()

X = df_model.drop('exam_score', axis=1)
y = df_model['exam_score']

# Identify numerical and categorical features for the model
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['category', 'object']).columns.tolist()

print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

In [ ]:
# Create preprocessing pipelines for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first') # drop='first' to avoid multicollinearity

# Create a preprocessor object using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Training and Evaluation

### 1. Linear Regression

In [ ]:
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', LinearRegression())])

lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Performance:")
print(f"RMSE: {rmse_lr:.2f}")
print(f"R²: {r2_lr:.2f}")

### 2. Random Forest Regressor

In [ ]:
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Regressor Performance:")
print(f"RMSE: {rmse_rf:.2f}")
print(f"R²: {r2_rf:.2f}")

*Note: The HTML reports RMSE of 8.52 (LR) / 6.95 (RF) and R² of 0.78 (LR) / 0.85 (RF). The actual calculated values will depend on the dataset, train/test split, and exact features used. The provided CSV is synthetic and these numbers are achievable with it.*

## Importância das Features (Exemplo do Random Forest)

In [ ]:
# Get feature names after one-hot encoding
feature_names_out = rf_pipeline.named_steps['preprocessor'].get_feature_names_out()

importances = rf_pipeline.named_steps['regressor'].feature_importances_
feature_importance_df = pd.DataFrame({'feature': feature_names_out, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

print("\nTop 10 Feature Importances from Random Forest:")
print(feature_importance_df.head(10))

plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(10), palette='mako') # Top 10
plt.title('Top 10 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

The HTML's feature importance chart shows `study_hours_per_day` as most important, followed by `attendance_percentage`, `mental_health_rating`, etc. The actual results from the model might vary slightly in order but should generally highlight similar features as strong predictors.